In [2]:
!pip install datasets torch transformers

   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/906.4 MB 2.3 MB/s eta 0:05:35^C
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.0/906.4 MB 2.3 MB/s eta 0:05:36
ERROR: Operation cancelled by user


In [3]:
!pip install datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Flatten, LSTM , GRU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import json
import random

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (8, 7)

2024-10-24 16:32:56.878394: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 16:32:56.887096: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 16:32:56.905558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 16:32:56.936775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 16:32:56.943842: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 16:32:56.967049: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
# Upload the kaggle.json to Colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d scodepy/customer-support-intent-dataset

Dataset URL: https://www.kaggle.com/datasets/scodepy/customer-support-intent-dataset
License(s): unknown
customer-support-intent-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip customer-support-intent-dataset.zip -d ./data/

Archive:  customer-support-intent-dataset.zip
replace ./data/Bitext_Sample_Customer_Service_Testing_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls data

In [12]:
df_train=pd.read_csv("data/Bitext_Sample_Customer_Service_Training_Dataset.csv")
df_test=pd.read_csv("data/Bitext_Sample_Customer_Service_Testing_Dataset.csv")
df_val=pd.read_csv("data/Bitext_Sample_Customer_Service_Validation_Dataset.csv")

In [ ]:
# train_path="/kaggle/input/customer-support-intent-dataset/Bitext_Sample_Customer_Service_Training_Dataset.csv"
# valid_path="/kaggle/input/customer-support-intent-dataset/Bitext_Sample_Customer_Service_Validation_Dataset.csv"
# test_data="/kaggle/input/customer-support-intent-dataset/Bitext_Sample_Customer_Service_Testing_Dataset.csv"
# df_train=pd.read_csv(train_path)
# df_val=pd.read_csv(valid_path)
# df_test=pd.read_csv(test_data)

# Exploratory Data Analysis

In [13]:
df_train.head()

,utterance,intent,category,tags
0,would it be possible to cancel the order I made?,cancel_order,ORDER,BIP
1,cancelling order,cancel_order,ORDER,BK
2,I need assistance canceling the last order I h...,cancel_order,ORDER,B
3,problem with canceling the order I made,cancel_order,ORDER,B
4,I don't know how to cancel the order I made,cancel_order,ORDER,B


In [14]:
df_train.shape

(6539, 4)

In [15]:
df_test.shape

(818, 4)

In [16]:
df_val.shape

(818, 4)

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6539 entries, 0 to 6538
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  6539 non-null   object
 1   intent     6539 non-null   object
 2   category   6539 non-null   object
 3   tags       6539 non-null   object
dtypes: object(4)
memory usage: 204.5+ KB


In [18]:
df_train['intent'].value_counts()

intent
get_invoice                 268
payment_issue               263
check_invoice               258
contact_customer_service    253
complaint                   252
review                      248
cancel_order                246
check_cancellation_fee      246
check_payment_methods       246
set_up_shipping_address     246
track_order                 245
place_order                 243
check_refund_policy         240
get_refund                  239
registration_problems       239
recover_password            239
track_refund                239
newsletter_subscription     238
create_account              237
delivery_options            236
delete_account              236
delivery_period             236
edit_account                233
change_order                232
switch_account              228
change_shipping_address     228
contact_human_agent         225
Name: count, dtype: int64

In [19]:
df_train["category"].value_counts()

category
ACCOUNT             1412
ORDER                966
REFUND               718
INVOICE              526
PAYMENT              509
FEEDBACK             500
CONTACT              478
SHIPPING_ADDRESS     474
DELIVERY             472
CANCELLATION_FEE     246
NEWSLETTER           238
Name: count, dtype: int64

# data preprocessing

In [20]:

def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [21]:
df_train['utterance']=df_train['utterance'].apply(preprocess_text)
df_test['utterance']=df_test['utterance'].apply(preprocess_text)
df_val['utterance']=df_val['utterance'].apply(preprocess_text)

In [22]:
train_texts = df_train['utterance'].tolist()
train_labels = df_train['intent'].tolist()

In [23]:
len(train_texts), len(train_labels)

(6539, 6539)

# MODELING

## Naive Bayes

In [27]:

# Ensure you have downloaded necessary NLTK resources
nltk.download('punkt')

# Tokenize the text (you can use NLTK's word_tokenize or simply split)
train_texts = [nltk.word_tokenize(text) for text in train_texts]

# Convert the tokenized texts to strings (as needed for vectorization)
train_texts = [" ".join(text) for text in train_texts]

# Vectorize the text using scikit-learn's CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_texts)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, train_labels, test_size=0.2, random_state=42)

# Train a Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = f1_score(y_test, y_pred, average = 'macro')
print(f"Accuracy: {accuracy:.2f}")

[nltk_data] Downloading package punkt to /home/ghannam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/ghannam/nltk_data'
    - '/home/ghannam/anaconda3/envs/mlflow/nltk_data'
    - '/home/ghannam/anaconda3/envs/mlflow/share/nltk_data'
    - '/home/ghannam/anaconda3/envs/mlflow/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [16]:
# Example test sentences to classify
new_texts = ["I need help with my order", "Thank you for your help"]

# Tokenize the new texts
new_texts_tokenized = [" ".join(nltk.word_tokenize(text)) for text in new_texts]

# Vectorize the new texts (using the same vectorizer that was fitted on the training data)
new_texts_vectorized = vectorizer.transform(new_texts_tokenized)

# Make predictions using the trained classifier
predictions = classifier.predict(new_texts_vectorized)

# Output the predictions
for text, label in zip(new_texts, predictions):
    print(f"Text: {text} --> Predicted Intent: {label}")

Text: I need help with my order --> Predicted Intent: cancel_order
Text: Thank you for your help --> Predicted Intent: review


## RNN

In [17]:
df_train.head()

,utterance,intent,category,tags
0,would it be possible to cancel the order i made,cancel_order,ORDER,BIP
1,cancelling order,cancel_order,ORDER,BK
2,i need assistance canceling the last order i h...,cancel_order,ORDER,B
3,problem with canceling the order i made,cancel_order,ORDER,B
4,i dont know how to cancel the order i made,cancel_order,ORDER,B


In [18]:
df_train.columns

Index(['utterance', 'intent', 'category', 'tags'], dtype='object')

In [19]:
test_texts = df_test['utterance'].values
test_labels = df_test['intent'].values
X_val = df_val["utterance"].to_list()
y_val = df_val["intent"].to_list()

In [20]:
len(X_val)

818

In [21]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
y_test = label_encoder.transform(test_labels)
y_val = label_encoder.transform(y_val)

In [22]:
len(train_texts), len(train_labels)

(6539, 6539)

In [23]:
tokenizer = Tokenizer(num_words=5000)  # Keep the top 5000 words
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
val_sequences = tokenizer.texts_to_sequences(X_val)

In [24]:
max_sequence_length = max([len(seq) for seq in train_sequences])
X_train = pad_sequences(train_sequences, maxlen=max_sequence_length)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
x_val = pad_sequences(val_sequences, maxlen=max_sequence_length)


In [25]:
len(x_val)

818

In [ ]:
# def preprocessing(data):
#     tokenizer = Tokenizer(num_words=5000)
#     tokenizer.fit_on_texts(data)
#     sequences = tokenizer.texts_to_sequences(data)
#     max_sequence_length = max([len(seq) for seq in sequences])
#     data = pad_sequences(sequences, maxlen=max_sequence_length)
#     return data

In [ ]:
# tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
# def preprocessing_gpt(data):
#     global max_sequence_length

#     # Calculate max_sequence_length only once
#     if max_sequence_length is None:
#         tokenizer.fit_on_texts(data)
#         sequences = tokenizer.texts_to_sequences(data)
#         max_sequence_length = max(len(seq) for seq in sequences)

#     # Reuse the same tokenizer and pad with the fixed max length
#     tokenizer = Tokenizer(num_words=5000)
#     tokenizer.fit_on_texts(data)
#     sequences = tokenizer.texts_to_sequences(data)
#     data = pad_sequences(sequences, maxlen=max_sequence_length)

#     return data, tokenizer

In [ ]:
# x_train, x_test, x_val = preprocessing1(train_texts,test_texts,X_val)

In [ ]:
# x_train = preprocessing_gpt(train_texts)
# x_test = preprocessing_gpt(test_texts)
# x_val = preprocessing_gpt(X_val)

In [ ]:
# x_train = preprocessing(train_texts)
# x_test = preprocessing(test_texts)
# x_val = preprocessing(X_val)

In [26]:
len(X_train)

6539

In [27]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=max_sequence_length))
model.add(SimpleRNN(128))
model.add(Dense(27, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [28]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.4061 - loss: 2.7250 - val_accuracy: 0.8435 - val_loss: 0.8520
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9278 - loss: 0.5183 - val_accuracy: 0.9609 - val_loss: 0.2474
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9785 - loss: 0.1517 - val_accuracy: 0.9731 - val_loss: 0.1382
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9939 - loss: 0.0662 - val_accuracy: 0.9804 - val_loss: 0.1063
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9961 - loss: 0.0442 - val_accuracy: 0.9841 - val_loss: 0.0776
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9987 - loss: 0.0227 - val_accuracy: 0.9878 - val_loss: 0.0662
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9994 - loss: 0.0149 - val_accuracy: 0.9890 - val_loss: 0.0510
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9998 - loss: 0.0105 - val_accu

In [32]:
import pickle as pk
#save tokinizer
pk.dump(tokenizer,open("tokenizer.pkl","wb"))

In [34]:
#save encoder
pk.dump(label_encoder,open("label_encoder.pkl","wb"))

In [35]:
model.save('model.h5') # save model
from tensorflow.keras.models import load_model
model=load_model('model.h5') # load model

In [ ]:
model.save("rnn_model.keras")

In [ ]:
model  = load_model("rnn_model.keras")

In [ ]:
with open('/content/intent_responses.json', 'r') as f:
    responses = json.load(f)

In [ ]:
def preprocess_input(input_text):
    input_text = input_text.lower()
    input_text = re.sub(r'\s+', ' ', input_text)  # Remove extra spaces
    input_text = re.sub(r'[^\w\s]', '', input_text)  # Remove punctuation

    # Tokenize and pad the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])  # Pass as a list
    input_sequence = pad_sequences(input_sequence, maxlen=20)  # Use a fixed max length

    return input_sequence

In [ ]:
def get_response(user_input):
    # Step 5: Preprocess the user input
    input_vector = preprocess_input(user_input)

    # Step 6: Predict the intent using the model
    prediction = model.predict(input_vector)  # Ensure input is in the right format

    # Step 7: Get the predicted intent
    intent_index = np.argmax(prediction)  # Assuming softmax output
    intents = list(responses.keys())
    predicted_intent = intents[intent_index]

    # Step 8: Fetch a random response for the predicted intent
    response = random.choice(responses[predicted_intent])

    return response

In [ ]:
get_response("I need help with my order")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


"Honored to assist! I'm clued in that you are unsure of how to check the status of your order with the order number {{Order Number}}. I'm here to help guide you through the process. To view the status of your order, you can visit the '{{Order Tracking}}' section on our website. This will provide you with real-time updates on the progress of your purchase. If you have any further questions or need additional assistance, please feel free to ask. It's my priority to ensure you have a seamless experience!"

In [ ]:
def chat():
    print("Chatbot: Hello! How can I assist you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = get_response(user_input)
        print("Chatbot:", response)


In [ ]:
chat()

Chatbot: Hello! How can I assist you today?
You: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Chatbot: Glad you contacted to us for assistance in canceling your {{Account Category}} account. I understand that you no longer find value in your current subscription and we value your feedback. To cancel your {{Account Category}} account, please follow these steps:

1. Log in to your account on our website.
2. Navigate to the account settings page.
3. Look for the option to manage your subscriptions.
4. Locate the {{Account Category}} account subscription and select the cancellation option.
5. Follow the prompts and confirm the cancellation of your {{Account Category}} account.

If you encounter any difficulties or have any questions during the cancellation process, don't hesitate to reach out to our customer support team. We are here to assist you every step of the way. We appreciate your business and hope to serve you again in the future.
You: i want to return my order
1/1 ━━━━━━━━━━━━━━━━

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train)
# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)
# X_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
# max_len = 100
# X_train = pad_sequences(X_train, maxlen=max_len)
# X_test = pad_sequences(X_test, maxlen=max_len)
# X_val = pad_sequences(X_val, maxlen=max_len)

In [ ]:
# label_encoder = LabelEncoder()
# y_train = label_encoder.fit_transform(y_train)
# y_test = label_encoder.transform(y_test)
# y_val = label_encoder.transform(y_val)

## FULLY CONNCTED NETWORK

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=100))
model.add(Flatten())
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(len(label_encoder.classes_), activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)

loss, acc = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", acc)

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0770 - loss: 3.2506 - val_accuracy: 0.2922 - val_loss: 2.7265
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4024 - loss: 2.3461 - val_accuracy: 0.7090 - val_loss: 1.2772
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8162 - loss: 0.9460 - val_accuracy: 0.9169 - val_loss: 0.5283
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9459 - loss: 0.3738 - val_accuracy: 0.9462 - val_loss: 0.2971
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9726 - loss: 0.1831 - val_accuracy: 0.9621 - val_loss: 0.2217
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9843 - loss: 0.1071 - val_accuracy: 0.9719 - val_loss: 0.1738
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9905 - loss: 0.0710 - val_accuracy: 0.9756 - val_loss: 0.1499
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9938 - loss: 0.0535 - val_accuracy: 0.

In [ ]:
y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)
y_pred = label_encoder.inverse_transform(y_pred)

print(y_pred)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
['cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'cancel_order' 'cancel_order' 'cancel_order'
 'cancel_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order' 'change_order'
 'change_order' 'change_order' 'change_order'

In [ ]:
results_df = pd.DataFrame({"Actual Intent": df_test['intent'], "Predicted Intent": y_pred})
print(results_df)

    Actual Intent Predicted Intent
0    cancel_order     cancel_order
1    cancel_order     cancel_order
2    cancel_order     cancel_order
3    cancel_order     cancel_order
4    cancel_order     cancel_order
..            ...              ...
813  track_refund     track_refund
814  track_refund     track_refund
815  track_refund     track_refund
816  track_refund     track_refund
817  track_refund     track_refund

[818 rows x 2 columns]


In [ ]:
mismatch = results_df[results_df['Actual Intent'] != results_df['Predicted Intent']]

print(mismatch)

               Actual Intent Predicted Intent
116   check_cancellation_fee      place_order
125   check_cancellation_fee     cancel_order
315           create_account   switch_account
317           create_account      place_order
332           create_account   delete_account
414          delivery_period        complaint
437          delivery_period      track_order
544  newsletter_subscription     edit_account
639    registration_problems     edit_account
644    registration_problems   create_account
742           switch_account   delete_account
773              track_order     cancel_order
775              track_order  delivery_period


## LSTM

In [ ]:

model = Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_sequence_length))

model.add(LSTM(64))

model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))

model.add(Dense(len(label_encoder.classes_), activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)
# Evaluate the model
loss, acc = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", acc)


Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.1104 - loss: 3.1749 - val_accuracy: 0.2861 - val_loss: 2.3149
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.4443 - loss: 1.8538 - val_accuracy: 0.8130 - val_loss: 0.9854
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8639 - loss: 0.7671 - val_accuracy: 0.9315 - val_loss: 0.4542
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9554 - loss: 0.3467 - val_accuracy: 0.9682 - val_loss: 0.2486
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9767 - loss: 0.1924 - val_accuracy: 0.9768 - val_loss: 0.1679
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9884 - loss: 0.1090 - val_accuracy: 0.9804 - val_loss: 0.1232
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9905 - loss: 0.0742 - val_accuracy: 0.9829 - val_loss: 0.1021
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9954 - loss: 0.0459 - val_accu

In [ ]:

model = Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length= max_sequence_length))

model.add(SimpleRNN(64))

model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))

model.add(Dense(len(label_encoder.classes_), activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)
loss, acc = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", acc)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


103/103 ━━━━━━━━━━━━━━━━━━━━ 25s 11ms/step - accuracy: 0.1058 - loss: 3.1491 - val_accuracy: 0.3460 - val_loss: 2.3905
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4278 - loss: 2.0303 - val_accuracy: 0.7262 - val_loss: 1.2157
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7938 - loss: 0.9594 - val_accuracy: 0.9205 - val_loss: 0.4653
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9489 - loss: 0.3187 - val_accuracy: 0.9633 - val_loss: 0.2040
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9879 - loss: 0.1085 - val_accuracy: 0.9731 - val_loss: 0.1297
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9929 - loss: 0.0578 - val_accuracy: 0.9743 - val_loss: 0.1130
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9954 - loss: 0.0377 - val_accuracy: 0.9792 - val_loss: 0.1060
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9987 - loss: 0.0201 - val_accuracy: 0.9768 - 

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=128)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_texts = df_train['utterance'].values
train_labels = df_train['intent'].values
val_texts = df_val['utterance'].values
val_labels = df_val['intent'].values

NameError: name 'df_train' is not defined

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer



In [ ]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

NameError: name 'train_labels' is not defined

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts and include labels
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=128)

# Add labels to the encodings
train_encodings['labels'] = train_labels
val_encodings['labels'] = y_val

# Convert encodings to datasets
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# Initialize the data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=27)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    report_to="none"  # Disabling W&B logging (optional)
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).